---
jupyter: julia-1.10
# engine: julia
---






# Operations on rows


In [ ]:
using DataFrames, PalmerPenguins
using Tidier
import DataFramesMeta as DFM

penguins = PalmerPenguins.load() |> DataFrame;
@slice_head(penguins, n = 10)

## Filtering (or: throwing lines away)

To filter a dataframe means keeping only the rows that satisfy a certain criteria (ie. a boolean condition).

To filter a dataframe in Tidier, we use the macro `@filter`. You can use it in the form


In [ ]:
@filter(penguins, species == "Adelie")

or without parentesis as in 


In [ ]:
@filter penguins species == "Adelie"

Notice that the columns are typed as if they were variables on the Julia environment. This is inspired by the `tidyverse` behaviour of data-masking: inside a tidyverse verb, the columns are taken as "statistical variables" that exist inside the dataframe as columns.

In DataFramesMeta, we have two macros for filtering: `@subset` and `@rsubset`. Use the first when you have some criteria that uses a whole column, for example:


In [ ]:
DFM.@subset penguins :body_mass_g .>= mean(skipmissing(:body_mass_g))

Notice the broadcast on >=. We need it because *each variable is interpreted as a vector (the whole column)*. Also, notice that we refer to columns as _symbols_ (i.e. we append `:` to it).

In the above example, we needed the whole column `body_mass_g` to take the mean and then filter the rows based on that. If, however, your filtering criteria only uses information about each row (without needing to see it in context of the whole column), then `@rsubset` (row subset) is easier to use: it interprets each columns as a value (not an array), so no broadcasting is needed:


In [ ]:
DFM.@rsubset penguins :species == "Adelie"

In both Tidier and DataFramesMeta, only the rows to which the criteria is `true` are returned. This means that `false` and `missing` are thrown away.

In pure DataFrames, we use the `subset` function, and the criteria is passed with the notation


In [ ]:
#| eval: false

subset(penguins, :column => boolean_function)

where `boolean_function` is a boolean (with possibly `missing` values) function on 1 variable (the `:column` you passed). Add the kwarg `skipmissing=true` if you want to get rid of missing values.

### Filtering with one criteria

Filtering all the rows with `species` == "Adelie".

::: {.panel-tabset}

## Tidier


In [ ]:
@filter penguins species == "Adelie"

## DataFramesMeta


In [ ]:
DFM.@rsubset penguins :species == "Adelie"

## DataFrames


In [ ]:
subset(penguins, :species => x -> x .== "Adelie", skipmissing=true)

:::

### Filtering with several criteria

Filtering all the rows with `species` == "Adelie", `sex` == "male" and `body_mass_g` > 4000.

::: {.panel-tabset}

## Tidier


In [ ]:
@filter penguins species == "Adelie" sex == "male" body_mass_g > 4000

## DataFramesMeta


In [ ]:
DFM.@rsubset penguins :species == "Adelie" :sex == "male" :body_mass_g > 4000

## DataFrames


In [ ]:
subset(
    penguins
    , [:species, :sex, :body_mass_g] => 
    (x, y, z) -> (x .== "Adelie") .& (y .== "male") .& (z .> 4000)
    ,skipmissing=true
)

:::


Filtering all the rows with `species` == "Adelie" OR `sex` == "male".

::: {.panel-tabset}

## Tidier


In [ ]:
@filter penguins (species == "Adelie") | (sex == "male")

## DataFramesMeta


In [ ]:
DFM.@rsubset penguins (:species == "Adelie") | (:sex == "male")

## DataFrames


In [ ]:
subset(penguins, [:species, :sex] => (x, y) -> (x .== "Adelie") .| (y .== "male"), skipmissing=true)

:::


Filtering all the rows where the `flipper_length_mm` is greater than the mean.

::: {.panel-tabset}

## Tidier


In [ ]:
@filter penguins flipper_length_mm > mean(skipmissing(flipper_length_mm))

## DataFramesMeta


In [ ]:
DFM.@subset penguins :flipper_length_mm .>= mean(skipmissing(:flipper_length_mm))

## DataFrames


In [ ]:
subset(penguins, :flipper_length_mm => x -> x .> mean(skipmissing(x)), skipmissing=true)

:::

### Filtering with a variable column name

Suppose the column you want to filter is a variable, let's say a symbol


In [ ]:
my_column = :species;

::: {.panel-tabset}

## Tidier


In [ ]:
@eval @filter penguins $my_column == "Adelie"

## DataFramesMeta


In [ ]:
DFM.@rsubset penguins $my_column == "Adelie"

## DataFrames


In [ ]:
subset(penguins, my_column => x -> x .== "Adelie")

:::

In case the column is a string


In [ ]:
my_column_string = "species";

instead of a symbol, we can write in the same way, just taking care in Tidier to convert it to a symbol

::: {.panel-tabset}

## Tidier


In [ ]:
@eval @filter penguins $(Symbol(my_column_string)) == "Adelie"

## DataFramesMeta


In [ ]:
DFM.@rsubset penguins $(my_column_string) == "Adelie"

## DataFrames


In [ ]:
subset(penguins, my_column_string => x -> x .== "Adelie")

:::

## Arranging

Arranging is when we reorder the rows of a dataframe according to some columns. The rows are first arranged by the first column, then by the second (if any), and so on. In Tidier, when we want to invert the ordering, just put the column name inside a `desc()` call.

### Arranging by one column

Arrange by `body_mass_g`.

::: {.panel-tabset}

## Tidier


In [ ]:
@arrange penguins body_mass_g

## DataFramesMeta


In [ ]:
DFM.@orderby penguins :body_mass_g

## DataFrames


In [ ]:
sort(penguins, :body_mass_g)

:::

### Arranging by two columns, with one reversed

First arrange by `island`, then by reversed `body_mass_g`.

::: {.panel-tabset}

## Tidier


In [ ]:
@arrange penguins island desc(body_mass_g)

## DataFramesMeta


In [ ]:
# works only when the reversed column is numeric?

DFM.@orderby penguins :island :body_mass_g .* -1

## DataFrames


In [ ]:
sort(penguins, [order(:island), order(:body_mass_g, rev=true)])

:::

### Arranging by one variable column

Let's arrange the data by the following column:


In [ ]:
my_arrange_column = :body_mass_g;

::: {.panel-tabset}

## Tidier


In [ ]:
@eval @arrange penguins $my_arrange_column

## DataFramesMeta


In [ ]:
DFM.@orderby penguins $my_arrange_column

## DataFrames


In [ ]:
sort(penguins, my_arrange_column)

:::